In [3]:
import csv
reader = csv.reader(open('datasets/data.csv'))
for row in reader:
    print(row)



['<!DOCTYPE html>']
['<html>']
['<head>']
['    <title>Kaggle: Your Home for Data Science</title>']
['    <meta charset="utf-8" />']
['    <meta name="robots" content="index', ' follow"/>']
['    <meta name="theme-color" content="#008ABC" />']
['    <link rel="dns-prefetch" href="https://www.google-analytics.com" /><link rel="dns-prefetch" href="https://stats.g.doubleclick.net" /><link rel="dns-prefetch" href="https://js.intercomcdn.com" /><link rel="preload" href="https://az416426.vo.msecnd.net/scripts/a/ai.0.js" as=script /><link rel="dns-prefetch" href="https://kaggle2.blob.core.windows.net" />']
['    <link href="/content/v/d420a040e581/kaggle/favicon.ico" rel="shortcut icon" type="image/x-icon" />']
['    <link rel="manifest" href="/static/json/manifest.json">']
['    <link href="//fonts.googleapis.com/css?family=Open+Sans:400', '300', '300italic', '400italic', '600', '600italic', '700', '700italic" rel=\'stylesheet\' type=\'text/css\'>']
['                    <link rel="styleshee